In [7]:
# Load libraries

import os
import torch
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
import shutil
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from ultralytics import YOLO
from sklearn.model_selection import train_test_split

In [8]:
# torch.cuda.empty_cache()  # Clears CUDA memory cache
# torch.cuda.reset_peak_memory_stats()
# torch.cuda.reset_accumulated_memory_stats()

### Prepare data for YOLO model input

In [9]:
# Base directories
data_directory = 'data'
output_directory = os.path.join('model_outputs_data', 'yolo_output')

# Paths to data
train_csv_path = os.path.join(data_directory, 'train.csv')
test_csv_path = os.path.join(data_directory, 'test.csv')
images_train_path = os.path.join(data_directory, 'images_train')
images_test_path = os.path.join(data_directory, 'images_test')

# Create necessary directories
def create_directories(path_list):
    for path in path_list:
        os.makedirs(path, exist_ok=True)

# Setup directories for training and validation
train_dir = os.path.join(output_directory, 'train')
val_dir = os.path.join(output_directory, 'val')
train_0_dir = os.path.join(train_dir, '0')
train_1_dir = os.path.join(train_dir, '1')
val_0_dir = os.path.join(val_dir, '0')
val_1_dir = os.path.join(val_dir, '1')

# Create all needed folders
create_directories([train_dir, val_dir, train_0_dir, train_1_dir, val_0_dir, val_1_dir])

# Method to copy images based on a dataframe, source directory, and target base directory
def copy_images(df, src_dir, target_base_dir):
    for idx, row in df.iterrows():
        file_name = f"{row['id']}.png"
        src_file_path = os.path.join(src_dir, file_name)
        if row['ground_truth'] == 1:
            dst_file_path = os.path.join(target_base_dir, '1', file_name)
        else:
            dst_file_path = os.path.join(target_base_dir, '0', file_name)
        shutil.copy(src_file_path, dst_file_path)

# Load the data
train_df = pd.read_csv(train_csv_path)
test_df = pd.read_csv(test_csv_path)

# Split train_df further to create a validation set or use test_df as validation
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Copy images according to the csv files
copy_images(train_df, images_train_path, train_dir)
copy_images(val_df, images_train_path, val_dir)  # If using part of train data as val
# copy_images(test_df, images_test_path, val_dir)  # Uncomment if test_df should be used as validation

print("Data organization complete.")


Data organization complete.


### Select device
I'm using my local GPU with Torch+CUDA for this because it is way faster than using CPU

In [10]:
# Select the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### Load and Train the model

In [11]:
# Load and Train the model
model = YOLO('pre_trained_models/yolov8x-cls.pt')  # Yolo8
# model = YOLO(os.path.join(output_directory, 'pre_trained_models/yolov8x-cls.pt'))  # Yolo8
model.to(device)

train_results = model.train(data=output_directory, epochs=10, seed=42, device=device)

New https://pypi.org/project/ultralytics/8.3.33 available  Update with 'pip install -U ultralytics'
engine\trainer: task=classify, mode=train, model=pre_trained_models/yolov8x-cls.pt, data=data\yolo_output, epochs=10, time=None, patience=100, batch=16, imgsz=224, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labe

KeyboardInterrupt: 

### Prediction and evaluation

In [ ]:
# Import best model
best_model = YOLO('runs/classify/train/weights/best.pt')

In [ ]:
# Predict
test_results = best_model.predict(source=images_test_path, save=True, device=device)

In [ ]:
# Prediction data preparation

image_id = []
image_prediction = []

for result in test_results:
    image_id.append(os.path.basename(result.path)[:-4])
    image_prediction.append(result.probs.top1)
    
# Loading testing csv
test_df = pd.read_csv(test_csv_path)

# Create a DataFrame from predictions
predictions_df = pd.DataFrame({
    'id': image_id,
    'predicted': image_prediction
})

# Convert IDs in test_df to string (to validate/match predictions_df)
test_df['id'] = test_df['id'].astype(str)

# Merge the predictions with the ground truths
results_df = pd.merge(test_df, predictions_df, on='id', how='left')

# Ensure no missing predictions
# results_df['predicted'].fillna(0, inplace=True)

In [ ]:
# Calculate evaluation metrics
accuracy = accuracy_score(results_df['ground_truth'], results_df['predicted'])
precision = precision_score(results_df['ground_truth'], results_df['predicted'], average='macro')
recall = recall_score(results_df['ground_truth'], results_df['predicted'], average='macro')
f1 = f1_score(results_df['ground_truth'], results_df['predicted'], average='macro')

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

In [ ]:
# Confusion matrix
cm = confusion_matrix(results_df['ground_truth'], results_df['predicted'])
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.show()